# <a id='toc1_'></a>[Analysis of Jobs that Requested and Used VRAM](#toc0_)
This notebook generates the analysis for jobs that requested some VRAM and run on partitions that their type is GPU and some GPU VRAM is used. It looks at these jobs, corresponding users, and PI groups.

**Table of contents**<a id='toc0_'></a>    
- [Analysis of Jobs that Requested and Used VRAM](#toc1_)    
  - [Setup](#toc1_1_)    
  - [Data Digestion and Preprocessing](#toc1_2_)    
  - [Narrowing Dataset to Relevant Partition](#toc1_3_)    
  - [Generate Baseline Metrics](#toc1_4_)    
  - [Job-Level Analysis](#toc1_5_)    
      - [Find most inefficient jobs with no VRAM constraints based on `requested_vram_efficiency_score`](#toc1_5_1_1_)    
    - [Generate all hoarding analysis metrics for jobs:](#toc1_5_2_)    
      - [Find most inefficient jobs hoarding node RAM based on `ram_hoarding_fraction_diff`](#toc1_5_2_1_)    
      - [Find most inefficient jobs hoarding CPU cores based on `core_hoarding_fraction_diff`](#toc1_5_2_2_)    
  - [User-Level Analysis](#toc1_6_)    
    - [Find Inefficient Users based on `requested_vram_efficiency_score`](#toc1_6_1_)    
    - [Generate all hoarding analysis metrics for users:](#toc1_6_2_)    
      - [Find most inefficient users hoarding node RAM based on `expected_value_ram_hoarding_fraction_diff`](#toc1_6_2_1_)    
      - [Find most inefficient users hoarding CPU cores based on `expected_value_core_hoarding_fraction_diff`](#toc1_6_2_2_)    
  - [PI Group Analysis](#toc1_7_)    
      - [Find Inefficient PIs based on `avg_requested_vram_efficiency_score`](#toc1_7_1_1_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## <a id='toc1_1_'></a>[Setup](#toc0_)

In [ ]:
# Import required modules
import sys
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Jupyter server should be run at the notebook directory, so the output of the following cell would be the project root:

In [ ]:
project_root = str(Path.cwd().resolve().parent.parent)
print(f"Project root: {project_root}")

In [ ]:
# Automatically reload modules before executing code (set this up BEFORE imports)
%load_ext autoreload
%autoreload 2

# Add project root to sys.path for module imports
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from src.analysis import ResourceHoarding
from src.analysis import efficiency_analysis as ea
from src.visualization import JobsWithMetricsVisualizer, UsersWithMetricsVisualizer
from src.config.enum_constants import ResourceHoardingDataFrameNameEnum

## <a id='toc1_2_'></a>[Data Digestion and Preprocessing](#toc0_)

In [ ]:
# Load the jobs DataFrame from DuckDB
preprocessed_jobs_df = ea.load_preprocessed_jobs_dataframe_from_duckdb(
    db_path=Path(project_root) / "data/slurm_data.db",
    table_name="Jobs",
)
display(preprocessed_jobs_df.head(10))
print(preprocessed_jobs_df.shape)

## <a id='toc1_3_'></a>[Narrowing Dataset to Relevant Partition](#toc0_)


In [ ]:
analyzer = ResourceHoarding(
    jobs_df=preprocessed_jobs_df
)

In [ ]:
filtered_jobs = analyzer.filter_jobs_for_analysis(
    gpu_mem_usage_filter={"min": 0, "inclusive": False},
    requested_vram_filter={"min": 0, "inclusive": False}
)
filtered_jobs

## <a id='toc1_4_'></a>[Generate Baseline Metrics](#toc0_)

In [ ]:
metrics_dict = analyzer.calculate_all_efficiency_metrics(filtered_jobs)

jobs_with_metrics = metrics_dict["jobs_with_efficiency_metrics"]
users_with_metrics = metrics_dict["users_with_efficiency_metrics"]
pi_accounts_with_metrics = metrics_dict["pi_accounts_with_efficiency_metrics"]

## <a id='toc1_5_'></a>[Job-Level Analysis](#toc0_)

In [ ]:
# Set option to display all columns
pd.set_option("display.max_columns", None)
# Display the DataFrame
display(jobs_with_metrics.head(10))
# To revert to default settings (optional)
pd.reset_option("display.max_columns")

print(f"Jobs found: {len(jobs_with_metrics)}")

#### <a id='toc1_5_1_1_'></a>[Find most inefficient jobs with no VRAM constraints based on `requested_vram_efficiency_score`](#toc0_)

In [ ]:
inefficient_jobs_vram_hours = analyzer.sort_and_filter_records_with_metrics(
    metrics_df_name_enum=ResourceHoardingDataFrameNameEnum.JOBS,
    sorting_key="requested_vram_efficiency_score",
    ascending=True,  # Sort by requested_vram_efficiency_score in ascending order
    filter_criteria={
        "alloc_vram_efficiency_score": {"max": -10, "inclusive": True},  # score threshold
    },
)
# Display top inefficient users by requested VRAM efficiency score
print("\nTop inefficient Jobs by Requested VRAM Efficiency Score:")
display(inefficient_jobs_vram_hours.head(10))

# Plot top inefficient jobs by requested VRAM efficiency score, with VRAM-hours as labels
jobs_with_metrics_visualizer = JobsWithMetricsVisualizer(inefficient_jobs_vram_hours.head(10))
jobs_with_metrics_visualizer.visualize(
    column="requested_vram_efficiency_score",
    bar_label_columns=["vram_hours", "allocated_vram"],
    figsize=(10, 6)
)

### <a id='toc1_5_2_'></a>[Generate all hoarding analysis metrics for jobs:](#toc0_)

In [ ]:
memory_hoarding_jobs = analyzer.calculate_node_resource_hoarding_for_jobs(filtered_jobs)

# Set option to display all columns
pd.set_option("display.max_columns", None)
# Display the DataFrame
display(memory_hoarding_jobs.head(10))
# To revert to default settings (optional)
pd.reset_option("display.max_columns")

print(f"Jobs found: {len(memory_hoarding_jobs)}")

#### <a id='toc1_5_2_1_'></a>[Find most inefficient jobs hoarding node RAM based on `ram_hoarding_fraction_diff`](#toc0_)

In [ ]:
inefficient_jobs_hoarding_ram = analyzer.sort_and_filter_records_with_metrics(
    metrics_df_name_enum=ResourceHoardingDataFrameNameEnum.JOBS_WITH_RESOURCE_HOARDING_METRICS,
    sorting_key="ram_hoarding_fraction_diff",
    ascending=False,  # Sort in descending order
    filter_criteria={"ram_hoarding_fraction_diff": {"min": 0, "inclusive": True}},
)
# Display top inefficient users by RAM hoarding fraction
print("\nTop inefficient Jobs by RAM hoarding fraction:")
display(inefficient_jobs_hoarding_ram.head(10))

# Plot top inefficient jobs by RAM hoarding fraction, with RAM hoarding fraction as labels
jobs_with_metrics_visualizer = JobsWithMetricsVisualizer(inefficient_jobs_hoarding_ram.head(10))
jobs_with_metrics_visualizer.visualize(
    column="ram_hoarding_fraction_diff",
    bar_label_columns=["cpu_mem_efficiency", "alloc_vram_efficiency"],
    figsize=(10, 6),
)

#### <a id='toc1_5_2_2_'></a>[Find most inefficient jobs hoarding CPU cores based on `core_hoarding_fraction_diff`](#toc0_)

In [ ]:
inefficient_jobs_hoarding_cpu_cores = analyzer.sort_and_filter_records_with_metrics(
    metrics_df_name_enum=ResourceHoardingDataFrameNameEnum.JOBS_WITH_RESOURCE_HOARDING_METRICS,
    sorting_key="core_hoarding_fraction_diff",
    ascending=False,  # Sort in descending order
    filter_criteria={"core_hoarding_fraction_diff": {"min": 0, "inclusive": True}},
)
# Display top inefficient users by CPU core hoarding fraction
print("\nTop inefficient Jobs by CPU core hoarding fraction:")
display(inefficient_jobs_hoarding_cpu_cores.head(10))

# Plot top inefficient jobs by CPU core hoarding fraction, with CPU core hoarding fraction as labels
jobs_with_metrics_visualizer = JobsWithMetricsVisualizer(inefficient_jobs_hoarding_cpu_cores.head(10))
jobs_with_metrics_visualizer.visualize(
    column="core_hoarding_fraction_diff",
    bar_label_columns=["ram_hoarding_fraction_diff", "alloc_vram_efficiency"],
    figsize=(10, 6),
)

## <a id='toc1_6_'></a>[User-Level Analysis](#toc0_)

In [ ]:
users_with_metrics

### <a id='toc1_6_1_'></a>[Find Inefficient Users based on `requested_vram_efficiency_score`](#toc0_)

In [ ]:
inefficient_users_avg_req_vram_eff_score = analyzer.sort_and_filter_records_with_metrics(
    metrics_df_name_enum=ResourceHoardingDataFrameNameEnum.USERS,
    sorting_key="avg_requested_vram_efficiency_score",
    ascending=True,  # Sort by avg_requested_vram_efficiency_score in ascending order
    filter_criteria={
        "avg_requested_vram_efficiency_score": {"max": -10, "inclusive": True},  # score threshold
        "job_count": {"min": 5, "inclusive": True}  # minimum job count threshold
    },
)
# Display top inefficient users by avg_requested_vram_efficiency_score
print("\nTop inefficient users by Avg Requested VRAM Efficiency Score:")
display(inefficient_users_avg_req_vram_eff_score.head(10))
print(len(inefficient_users_avg_req_vram_eff_score), "inefficient users found.")

# Plot top inefficient users by Avg Requested VRAM Efficiency Score, with avg_requested_vram_efficiency_score as labels
users_with_metrics_visualizer = UsersWithMetricsVisualizer(inefficient_users_avg_req_vram_eff_score.head(10))
users_with_metrics_visualizer.visualize(
    column="avg_requested_vram_efficiency_score", bar_label_columns=["vram_hours", "job_count"], figsize=(10, 6)
)

In [ ]:
# Distribution of Avg Requested VRAM Efficiency Score (actual values; all are <= 0)
# We keep scores as-is (negative or zero) and construct bins that respect the skew while
# still giving higher resolution near zero using log-spaced absolute values mapped back to negatives.
scores = inefficient_users_avg_req_vram_eff_score["avg_requested_vram_efficiency_score"].dropna()
print(len(scores), "scores found for plotting.")
if scores.empty:
    print("No scores to plot.")
else:
    # If all scores are exactly zero, a histogram is not informative
    if (scores != 0).sum() == 0:
        print("All scores are zero; histogram not informative.")
    else:
        import numpy as np
        fig, ax = plt.subplots(figsize=(8, 5))

        # Separate negatives (expected) from zeros
        neg_scores = scores[scores < 0]
        zero_scores = scores[scores == 0]

        min_abs = None  # track smallest non-zero absolute value for symlog threshold

        # Build bins: if we have negative values, create log-spaced absolute edges then map back
        if not neg_scores.empty:
            n_bins = 100
            min_abs = neg_scores.abs().min()
            max_abs = neg_scores.abs().max()
            if min_abs == max_abs:
                # Degenerate case: all negative values identical -> fall back to linear bins
                bins = np.linspace(neg_scores.min(), 0, 20)
            else:
                abs_edges = np.logspace(np.log10(min_abs), np.log10(max_abs), n_bins)
                # Convert absolute edges to negative edges (descending), then append 0 as the last edge
                neg_edges = -abs_edges[::-1]
                bins = np.unique(np.concatenate([neg_edges, [0]]))  # ensure strictly increasing
        else:
            # No negative values (only zeros) already handled earlier; fallback just in case
            bins = 3

        sns.histplot(scores, bins=bins, color="#1f77b4", ax=ax)
        ax.set_xlabel("Avg Requested VRAM Efficiency Score (<= 0)")
        ax.set_ylabel("Count")
        ax.set_title("Distribution of Avg Requested VRAM Efficiency Scores (Actual Values, Log X)")

        # Apply symmetrical log scale to x-axis to compress the long negative tail while keeping zero.
        # linthresh defines the range around zero that stays linear; choose smallest non-zero magnitude.
        if min_abs is not None and min_abs > 0:
            linthresh = min_abs
        else:
            linthresh = 1e-6  # fallback small threshold
        ax.set_xscale('symlog', linthresh=linthresh, linscale=1.0, base=10)

        # Annotation: counts (negative & zero) and total
        neg_count = (scores < 0).sum()
        zero_count = (scores == 0).sum()
        total = len(scores)
        ax.text(
            0.98,
            0.95,
            ("Counts:\n" f"Negative: {neg_count}\n" f"Zero: {zero_count}\n" f"# of Users: {total}"),
            transform=ax.transAxes,
            ha="right",
            va="top",
            fontsize=9,
            bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.9),
        )

        # Cumulative distribution (CDF) over actual score values
        counts, bin_edges = np.histogram(scores, bins=bins)
        cdf = np.cumsum(counts) / counts.sum()
        mids = (bin_edges[1:] + bin_edges[:-1]) / 2
        ax2 = ax.twinx()
        ax2.plot(mids, cdf, color="crimson", marker="o", linestyle="-", linewidth=1, markersize=3)
        ax2.set_ylim(0, 1)
        ax2.set_ylabel("Cumulative Fraction", color="crimson")
        ax2.tick_params(axis='y', colors='crimson')

        # Ensure x-axis spans to (slightly) include zero for clarity
        left, right = ax.get_xlim()
        if right < 0:
            ax.set_xlim(left, 0)

        plt.tight_layout()
        plt.show()

# Notes:
# - We plot the actual (negative/zero) scores instead of absolute values.
# - symlog x-scale provides a log-like compression for large negative magnitudes while keeping zero.
# - linthresh picks the smallest non-zero magnitude so near-zero structure is visible.
# - CDF is computed over actual values to show accumulation from most negative toward zero.


### <a id='toc1_6_2_'></a>[Generate all hoarding analysis metrics for users:](#toc0_)

In [ ]:
memory_hoarding_users = analyzer.calculate_node_resource_hoarding_for_users(filtered_jobs)
display(memory_hoarding_users)

#### <a id='toc1_6_2_1_'></a>[Find most inefficient users hoarding node RAM based on `expected_value_ram_hoarding_fraction_diff`](#toc0_)

In [ ]:
inefficient_users_hoarding_ram = analyzer.sort_and_filter_records_with_metrics(
    metrics_df_name_enum=ResourceHoardingDataFrameNameEnum.USERS_WITH_RESOURCE_HOARDING_METRICS,
    sorting_key="expected_value_ram_hoarding_fraction_diff",
    ascending=False,  # Sort in descending order
    filter_criteria={"expected_value_ram_hoarding_fraction_diff": {"min": 0, "inclusive": True}},
)
# Display top inefficient users by RAM hoarding fraction

print("\nTop inefficient Users by RAM hoarding fraction:")
display(inefficient_users_hoarding_ram.head(10))

# Plot top inefficient users by RAM hoarding fraction, with RAM hoarding fraction as labels
users_with_metrics_visualizer = UsersWithMetricsVisualizer(inefficient_users_hoarding_ram.head(10))
users_with_metrics_visualizer.visualize(
    column="expected_value_ram_hoarding_fraction_diff",
    bar_label_columns=[
        "expected_value_alloc_vram_efficiency",
    ],
    figsize=(10, 6),
)

#### <a id='toc1_6_2_2_'></a>[Find most inefficient users hoarding CPU cores based on `expected_value_core_hoarding_fraction_diff`](#toc0_)

In [ ]:
inefficient_users_hoarding_cpu_cores = analyzer.sort_and_filter_records_with_metrics(
    metrics_df_name_enum=ResourceHoardingDataFrameNameEnum.USERS_WITH_RESOURCE_HOARDING_METRICS,
    sorting_key="expected_value_core_hoarding_fraction_diff",
    ascending=False,  # Sort in descending order
    filter_criteria={"expected_value_core_hoarding_fraction_diff": {"min": 0, "inclusive": True}},
)
# Display top inefficient users by CPU core hoarding fraction

print("\nTop inefficient Users by CPU core hoarding fraction:")
display(inefficient_users_hoarding_cpu_cores.head(10))

# Plot top inefficient users by CPU core hoarding fraction, with CPU core hoarding fraction as labels
users_with_metrics_visualizer = UsersWithMetricsVisualizer(inefficient_users_hoarding_cpu_cores.head(10))
users_with_metrics_visualizer.visualize(
    column="expected_value_core_hoarding_fraction_diff",
    bar_label_columns=[
        "expected_value_alloc_vram_efficiency",
    ],
    figsize=(10, 6),
)

## <a id='toc1_7_'></a>[PI Group Analysis](#toc0_)

In [ ]:
pi_accounts_with_metrics

#### <a id='toc1_7_1_1_'></a>[Find Inefficient PIs based on `avg_requested_vram_efficiency_score`](#toc0_)

In [ ]:
from matplotlib.transforms import blended_transform_factory

inefficient_pis_avg_req_vram_eff_score = analyzer.sort_and_filter_records_with_metrics(
    metrics_df_name_enum=ResourceHoardingDataFrameNameEnum.PI_GROUPS,
    sorting_key="avg_requested_vram_efficiency_score",
    ascending=True,  # more negative first
    filter_criteria={
        "avg_requested_vram_efficiency_score": {"max": -10, "inclusive": True},
        "job_count": {"min": 5, "inclusive": True},  # Minimum number of jobs to consider a PI account
    },
)
# Display top inefficient PI groups by Avg Requested VRAM Efficiency Score
print("\nTop inefficient PI Groups by Avg Requested VRAM Efficiency Score:")
display(inefficient_pis_avg_req_vram_eff_score.head(10))

# Take top N
top_pi_accounts = inefficient_pis_avg_req_vram_eff_score.head(10)
pi_accounts = top_pi_accounts["pi_account"].tolist()
user_counts = top_pi_accounts.get("user_count", ["-"] * len(top_pi_accounts)).tolist()

xmin = top_pi_accounts["avg_requested_vram_efficiency_score"].min()
# If the minimum value is negative, we need to adjust the heights of the bars
# to ensure they start from zero for better visualization.
# This is particularly useful for metrics like allocated VRAM efficiency score.
if xmin < 0:
    col_heights = pd.Series(
        [abs(xmin)] * len(top_pi_accounts["avg_requested_vram_efficiency_score"]),
        index=top_pi_accounts["avg_requested_vram_efficiency_score"].index
    ) - abs(top_pi_accounts["avg_requested_vram_efficiency_score"])
    print(f"Minimum value for \"avg_requested_vram_efficiency_score\": {xmin}")
else:
    col_heights = top_pi_accounts["avg_requested_vram_efficiency_score"]

plt.figure(figsize=(10, 6))
plot_df = pd.DataFrame({
    "col_height": col_heights.to_numpy(),
    "job_hours": top_pi_accounts["pi_acc_job_hours"],
    "pi_account": top_pi_accounts["pi_account"]
})
ax = sns.barplot(
    y=top_pi_accounts["pi_account"].tolist(),
    x=col_heights.tolist(),
    order=top_pi_accounts["pi_account"].tolist(),
    orient="h",
    palette="Blues_r",
    hue=top_pi_accounts["pi_account"].tolist()
)

# We'll replace the default tick labels with custom two-line labels placed OUTSIDE the left spine.
ax.set_yticks(range(len(pi_accounts)))
ax.set_yticklabels([])  # clear built-in labels
ax.set_ylabel("PI Account")

transform = blended_transform_factory(ax.transAxes, ax.transData)  # x in axes fraction, y in data coords
x_outside = -0.02  # negative x fraction places text just left of spine; adjust if needed
line_gap = 0.4  # vertical separation between the two lines
for y_pos, (pi, uc) in enumerate(zip(pi_accounts, user_counts, strict=True)):
    # First line (PI account) slightly above center
    ax.text(
        x_outside,
        y_pos - line_gap / 2,
        pi,
        ha="right",
        va="center",
        transform=transform,
        fontsize=10,
        clip_on=False,
    )
    # Second line (Users) slightly below center
    ax.text(
        x_outside,
        y_pos + line_gap / 2,
        f"# of Users: {uc}",
        ha="right",
        va="center",
        transform=transform,
        fontsize=9,
        color="dimgray",
        clip_on=False,
    )

# Y-axis label: place further left than custom tick labels.
ax.set_ylabel("PI Account", rotation=90, labelpad=20)
# Position the label using axes fraction (x< x_outside)
ax.yaxis.set_label_coords(x_outside - 0.30, 0.5) 

# Hide y-axis tick labels (already blank) but keep small outward ticks if desired
ax.tick_params(axis='y', which='both', direction='out', length=4, pad=2)
plt.subplots_adjust(left=0.7)

plt.xlabel("Average Requested VRAM Efficiency Score")
plt.title("Top Inefficient PI Accounts by Average Requested VRAM Efficiency Score")

xmax = top_pi_accounts["avg_requested_vram_efficiency_score"].max()

# Set x-axis limit to 1.6 times the maximum value
# This ensures that the bars do not touch the right edge of the plot
xlim_multiplier = 1.6
xlim = abs(xmin) * xlim_multiplier if xmin < 0 else (xmax * xlim_multiplier if xmax > 0 else 1)
ax.set_xlim(0, xlim)
# If the minimum value is negative, we need to adjust the x-ticks accordingly
if xmin < 0:
    num_xticks = max(4, min(12, int(abs(xmin) // (xlim * 0.10)) + 1))
    xticks = np.linspace(xmin, 0, num=num_xticks)
    ax.set_xticks([abs(xmin) - abs(val) for val in xticks])
    ax.set_xticklabels([f"{val:.2f}" if -1 < val < 1 else f"{val:.0f}" for val in xticks], rotation=45)
for i, (vram_hours, pi_acc_job_hours, avg_req_vram_eff_score) in enumerate(
    zip(
        top_pi_accounts["pi_acc_vram_hours"],
        top_pi_accounts["pi_acc_job_hours"],
        col_heights,
        strict=True,
    )
):
    xpos = min(avg_req_vram_eff_score + xlim * 0.02, xlim * 0.92)
    ax.text(
        xpos,
        i,
        f"VRAM-Hours: {vram_hours:.2f}\nJob Hours: {pi_acc_job_hours:.2f}",
        va="center",
        ha="left",
        fontsize=9,
        color="black",
        clip_on=False,
    )

ax.tick_params(axis='y', which='both', direction='out', length=4, pad=2)
plt.tight_layout()
plt.show()
